In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:

import requests
import sys
import time
import os
from datetime import datetime


In [ ]:
Primary_HOST = "http://" + os.getenv('Primary_IP') + ":82"
Secondary_HOST = "http://" + os.getenv('Secondary_IP') + ":82"
Primary_PASSWORD = os.getenv('APP_PASSWORD')
Secondary_PASSWORD = os.getenv('APP_PASSWORD')
CHECK_INTERVAL = os.getenv('CHECK_INTERVAL', '300')

In [5]:
def log(message):
    """Print timestamped log message"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {message}")

In [6]:
def auth_pihole(base_url, password):
    if not password:
        log(f"ERROR: No password provided for {base_url}")
        return None
    
    try:
        #log(f"Logging into {base_url}.")
        response = requests.post(
            f"{base_url}/api/auth",
            json={"password": password},
            timeout=10
        )
        response.raise_for_status()
        data = response.json()
        
        if "session" in data:
            #log(f"Logged into {base_url}.")
            return {
                "base_url":base_url,
                "sid": data["session"]["sid"],
                "csrf": data["session"]["csrf"],
            }
        else:
            log(f"ERROR: Authentication failed for {base_url}")
            return None
    except requests.exceptions.RequestException as e:
        log(f"ERROR: Could not connect to {base_url}: {e}")
        return False
    

In [7]:
def deauth_pihole(session):
    if not session:
        log("DEBUG: No session to logout from")
        return
    
    try:
        #log(f"Logging out from {session['base_url']}...")
        response = requests.delete(
            f"{session['base_url']}/api/auth",
            headers={
                "X-FTL-SID": session["sid"],
                "X-FTL-CSRF": session["csrf"]
            },
            timeout=10
        )
        response.raise_for_status()
        #log(f"Logged out from {session['base_url']}")
        return True
    except requests.exceptions.RequestException as e:
        log(f"WARNING: Could not logout from {session['base_url']}: {e}")
        return False

In [8]:
def get_dhcp_status(session):
    try:
        response = requests.get(
            f"{session['base_url']}/api/config",
            headers={
                "X-FTL-SID": session["sid"],
                "X-FTL-CSRF": session["csrf"]
            },
            timeout=10
        )
        response.raise_for_status()
        data = response.json()
        
        if "config" in data and "dhcp" in data["config"]:
            return data["config"]["dhcp"].get("active", False)
        return False
    except requests.exceptions.RequestException as e:
        log(f"ERROR: Could not get DHCP status from {session['base_url']}: {e}")
        return None

In [9]:
def set_dhcp_status(session, active):
    ## Active is boolean true or false
    try:
        response = requests.patch(
            f"{session['base_url']}/api/config",
            headers={
                "X-FTL-SID": session["sid"],
                "X-FTL-CSRF": session["csrf"],
                "Content-Type": "application/json"
            },
            json={
                "config": {
                    "dhcp": {
                        "active": active
                    }
                }
            },
            timeout=10
        )
        response.raise_for_status()
        data = response.json()
        
        if "error" in data:
            log(f"ERROR: Failed to set DHCP on {session['base_url']}: {data['error']}")
            return False
        return True
    except requests.exceptions.RequestException as e:
        log(f"ERROR: Could not set DHCP status on {session['base_url']}: {e}")
        return False

In [ ]:
def Check_and_Set_DHCP(host, password, active):
    ## active is a boolean passed into set_dhcp_Status(), determining failover or failback
    session = auth_pihole(host,password)
    if session == False:
        log(f"Unable to reach {host}.")
        return False
    try:
        dhcp_status = get_dhcp_status(session)

        if dhcp_status == None:
            log(f"Unable to see if DHCP running on {session['base_url']}")
            if set_dhcp_status(session, active) == False:
                log(f"Tried making DHCP active:{active} on {session['base_url']} unsuccessfully.")
                return False
            else:
                log(f"Made DHCP active:{active} on {session['base_url']} successfully.")
                return True
        elif dhcp_status == active:
            log(f"DHCP already active:{active} on {session['base_url']}.")
            return True
        elif dhcp_status != active:
            if set_dhcp_status(session, active) == False:
                log(f"Tried making DHCP active:{active} on {session['base_url']} unsuccessfully.")
                return False
            else:
                log(f"Made DHCP active:{active} on {session['base_url']} successfully.")
                return True
    finally:
        deauth_pihole(session)
 

In [11]:
def updater():
    primary_check_and_set = Check_and_Set_DHCP(Primary_HOST,Primary_PASSWORD, True)
    if primary_check_and_set == False: #Primary
        secondary_check_and_set = Check_and_Set_DHCP(Secondary_HOST,Secondary_PASSWORD, True)
        if secondary_check_and_set == False: #Secondary
            log(f"Unable to Enable Primary DHCP or Secondary DHCP")
            return 0
        elif secondary_check_and_set == True:
            log(f"Unable to Enable Primary DHCP, Secondary DHCP enabled.") 
            return 1
    elif primary_check_and_set == True:
        secondary_check_and_set = Check_and_Set_DHCP(Secondary_HOST,Secondary_PASSWORD, False)
        if secondary_check_and_set == True:
            log("Primary DHCP Enabled, Secondary DHCP Disabled.")
            return 1
        elif secondary_check_and_set == False:
            log("Primary DHCP Enabled, Unable to confirm status of secondary.")                          
            return 1




In [13]:
def main():
    """Initialize and start monitoring loop"""
    log("Starting Pi-hole DHCP failover monitor...")
    
    # Validate configuration
    if not Primary_HOST or not Secondary_HOST:
        log("CRITICAL: Pi-hole IP addresses must be set!")
        log("Set either Primary_IP or Secondary_IP environment variables")
        log(f"Current values: Primary_HOST={Primary_HOST}, Secondary_HOST={Secondary_HOST}")
        sys.exit(1)
    
    if not Primary_HOST or not Secondary_PASSWORD:
        log("WARNING: Passwords not set. Using empty passwords.")
    
    log(f"Configuration:")
    log(f"  Primary: {Primary_HOST}")
    log(f"  Secondary: {Secondary_HOST}")
    log(f"  Check interval: {CHECK_INTERVAL} seconds")
    
    # Main monitoring loop
    while True:
        try:
            exit_code = updater()
            if exit_code != 0:
                log(f"Check completed with exit code {exit_code}")
        except Exception as e:
            log(f"ERROR during check: {e}")
        
        log(f"Waiting {CHECK_INTERVAL} seconds until next check...")
        time.sleep(int(CHECK_INTERVAL))

    

In [ ]:
if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        log("Interrupted by user - shutting down")
        sys.exit(0)

[2025-11-01 22:33:04] Starting Pi-hole DHCP failover monitor...
[2025-11-01 22:33:04] Configuration:
[2025-11-01 22:33:04]   Primary: http://192.168.0.2:82
[2025-11-01 22:33:04]   Secondary: http://192.168.0.3:82
[2025-11-01 22:33:04]   Check interval: 300 seconds
[2025-11-01 22:33:04] DHCP already active:True on http://192.168.0.2:82.
[2025-11-01 22:33:07] ERROR: Could not connect to http://192.168.0.3:82: HTTPConnectionPool(host='192.168.0.3', port=82): Max retries exceeded with url: /api/auth (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002E58F4FE360>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
[2025-11-01 22:33:07] ERROR during check: 'bool' object is not subscriptable
[2025-11-01 22:33:07] Waiting 300 seconds until next check...
[2025-11-01 22:38:07] DHCP already active:True on http://192.168.0.2:82.
[2025-11-01 22:38:09] Made DHCP active:False on http://19